<a href="https://colab.research.google.com/github/sithin42/INT-PROSTATE-Contour-Stability/blob/main/RadiomicsFeatureExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

ROOT_PATH = "./"
#Loading the example data from github
if IN_COLAB:
  ROOT_PATH = "./INT-PROSTATE-Contour-Stability"
  !git clone https://github.com/sithin42/INT-PROSTATE-Contour-Stability.git
  import sys
  sys.path.append(ROOT_PATH)
  

In [ ]:
#Requirements 
!pip install torchio
!pip install SimpleITK
!pip install pyradiomics
!pip install pandas


In [ ]:
import os
import SimpleITK as sitk
from tqdm import tqdm
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import radiomics
import torchio as tio
import pandas as pd
from utils import ContourInPlaneAug, ContourOutPlaneAug, vol_dice_score, get_aug_fn

from ipywidgets import widgets, interact
import matplotlib.pyplot as plt

In [ ]:
import logging
# set level for all classes
logger = logging.getLogger("radiomics")
logger.setLevel(logging.ERROR)#To ignore feature warnings

# Data IO

In [ ]:
PID = "PCAMPMRI-00002"

img_path = os.path.join(ROOT_PATH,"data",PID,"image.nii.gz")
mask_path = os.path.join(ROOT_PATH,"data",PID,"mask.nii.gz")
  
sub = tio.Subject(img=tio.ScalarImage(img_path),mask=tio.LabelMap(mask_path))
sub.plot()#Fast Visualization

In [ ]:
#Interactive Visualization

sitk_img = sitk.ReadImage(img_path)
sitk_mask = sitk.ReadImage(mask_path)

img_arr = sitk.GetArrayFromImage(sitk_img)#Z,X,Y
mask_arr = sitk.GetArrayFromImage(sitk_mask)

spacing_W, spacing_H, _ = sitk_img.GetSpacing()

def visualize(i):
    
    plt.imshow(img_arr[i],cmap='gray')
    if mask_arr[i].sum()>0:
        plt.contour(mask_arr[i])
    plt.show()
    
interact(visualize, i=widgets.IntSlider(len(mask_arr)//2,0,len(mask_arr),1))

#Classical Radiomics Feature Extraction

In [ ]:
PATIENT_IDS = ["PCAMPMRI-00001","PCAMPMRI-00002"]
PARAM_SETTINGS = os.path.join(ROOT_PATH,"paramSettings/StudySettings3D.yaml")

In [ ]:
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(PARAM_SETTINGS,verbosity=False)

In [ ]:
def extract_features(pids):

  features = []

  pbar = tqdm(range(len(PATIENT_IDS)), desc="Extracting Features")

  for id in pids:

    img_path = os.path.join(ROOT_PATH,"data",id,"image.nii.gz")
    mask_path = os.path.join(ROOT_PATH,"data",id,"mask.nii.gz")

    featureVector = extractor.execute(img_path,mask_path)

    featureVector["id"] = id
    featureVector["dice"] = 1.0 #because this is the ground truth ROI
    featureVector["judge"] = 0

    features.append(featureVector)

    pbar.update()

  df = pd.DataFrame(features)

  return df


In [ ]:
df = extract_features(PATIENT_IDS)
if not os.path.exists("./results"):
  os.makedirs("./results")
df.to_csv("./results/org_feats.csv")
df.head()

#In-Silico Contour Generation & Feature Extraction

In [ ]:

AUG_COUNT = 15
AUG_TYPE = "in_plane"
BIAS_TYPE = "random"

IN_AUG_PARAMS = {'w_spacing':None,"h_spacing":None,'w_stdMM':2.7,'h_stdMM':2.7, 'angle':5,'bias_type':None}
OUT_AUG_PARAMS = {'scale_a':0.6,'scale_b':0.8,'angle':5,'delta_z':2}

PATIENT_IDS = ["PCAMPMRI-00001","PCAMPMRI-00002"]
PARAM_SETTINGS = os.path.join(ROOT_PATH,"paramSettings/StudySettings3D.yaml")

In [ ]:
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(PARAM_SETTINGS,verbosity=False)

In [ ]:
def extract_aug_features(pids, aug_type, bias_type, aug_count):
    
    features = []

    pbar = tqdm(range(aug_count*len(pids)), desc="Extracting Features", position=0)

    for pid in pids:

      img = sitk.ReadImage(os.path.join(ROOT_PATH,"data",pid,"image.nii.gz"))
      mask = sitk.ReadImage(os.path.join(ROOT_PATH,"data",pid,"mask.nii.gz"))

      img_arr = sitk.GetArrayFromImage(img)
      mask_arr = sitk.GetArrayFromImage(mask)

      aug_fn = get_aug_fn(aug_type, bias_type, img.GetSpacing(), IN_AUG_PARAMS, OUT_AUG_PARAMS)
      subject = tio.Subject(img=tio.ScalarImage(tensor=img_arr[np.newaxis,...]),mask=tio.LabelMap(tensor=mask_arr[np.newaxis,...]))

      for i in range(aug_count):

          pbar.set_description(f"Extracting Features {pid} - Synthetic ROI #{i+1}")

          aug_subject = aug_fn(subject)
          aug_mask_arr = aug_subject["mask"]["data"][0].numpy()

          dice = vol_dice_score(aug_mask_arr,mask_arr)

          aug_mask = sitk.GetImageFromArray(aug_mask_arr)
          aug_mask.SetSpacing(img.GetSpacing())
          aug_mask.SetOrigin(img.GetOrigin())

          featureVector = extractor.execute(img,aug_mask)

          featureVector['id'] = pid
          featureVector['dice'] = dice
          featureVector['judge'] = i+1

          features.append(featureVector)
          pbar.update()
            
                    
    aug_df = pd.DataFrame(features)

    return aug_df


In [ ]:
aug_df = extract_aug_features(PATIENT_IDS,AUG_TYPE,BIAS_TYPE,AUG_COUNT)

if not os.path.exists("./results"):
  os.makedirs("./results")
aug_df.to_csv("./results/aug_feats.csv")
aug_df.head()